# Lezione 11 - Fit con minimi quadrati

## Il metodo dei minimi quadrati
Il metodo dei minimi quadrati è un'alternativa a quello di maximum likelihood per la stima dei parametri di un modello. In questo caso, i parametri vengono scelti in modo tale da minimizzare la distanza tra il modello e i dati, secondo una metrica definita dalla media della deviazione standard delle misure rispetto al modello.

### Esempio
Per determinare la media di un set di misure $ x_{i} $ si deve minimizzare la funzione

$$ Q^2 = \sum_{i = 1}^{N}{\left(\frac{x_{i} - \mu}{\sigma}\right)} $$

### Caso $ y = \phi(x) $
La stessa metrica viene impiegata per il fit del modello con i dati. Siano date $ N $ coppie di misure indipendenti $ (x_{i}, y_{i}) $, tali che:
- L'incertezza su $ x_{i} $ è trascurabile o nulla
- L'incertezza su $ y_{i} $ è $ \sigma_{i} $
- Le variabili $ x_{i} $ e $ y_{i} $ sono legati dalla relazione $ y = \phi(x, \theta) $, dove $ \theta $ è il parametro da determinare

Si vuole quindi minimizzare la funzione

$$ Q^2(\theta) = \sum_{i = 1}^{N}{\left(\frac{y_{i} - \phi(x_{i}, \theta)}{\sigma_{i}}\right)^2} \implies \frac{\partial Q^2(\theta)}{\partial \theta} = 0 $$

### Proprietà del metodo dei minimi quadrati
- **Teorema di Gauss-Markov**: e gli scarti $ \epsilon_{i} = y_{i} - \phi(x_{i}, \theta) $ hanno media nulla e varianza costante e finita che non dipende da $ y $ e se $ \phi(x_{i}, \theta) $ è lineare nei parametri $ \theta $, allora
    1. Lo stimatore dei minimi quadrati è unbiased
    2. Tra tutti gli stimatori lineari unbiased è quello con la varianza minima

- Se i residui $ \epsilon_{i} $ sono distribuiti secondo una gaussiana, il minimo della funzione $ Q^2(\theta) $ è distribuito secondo $ \chi^2(N - k) $, dove $ N $ è il numero di coppie di dati e $ k $ è il numero di parametri

### Modello lineare nei parametri
Nel caso in cui il modello sia lineare **nei parametri**, cioé

$$ \phi(x, \theta) = \sum_{i = 1}^{N}{\theta_{i} \cdot h_{i}(x)} $$

allora l'equazione di minimizzazione può essere risolta analiticamente.

## Fit con minimi quadrati
Per eseguire i fit, si utilizza la libreria `iminuit`, che applica automaticamente il metodo dei minimi quadrati per adattare un modello ai dati. L'operazione di fit viene eseguita nel modo seguente:

```Python
from iminuit import Minuit
from iminuit.cost import LeastSquares

least_squares = LeastSquares(x_coord, y_coord, sigma_y, model)
my_minuit = Minuit(least_squares, m = 0, q = 0) # Valori iniziali per m e q
my_minuit.migrad() # Determina il minimo di Q^2
my_minuit.hesse() # Determina le incertezze
```

### Analisi dei risultati del fit
Per verificare che la minimizzazione abbia avuto successo

```Python
is_valid = my_minuit.valid()
print(f"Success: {is_valid}")
```

Per visualizzare il valore di $ Q^2 $ e il numero di gradi di libertà

```Python
Q_squared = my_minuit.fval
N_dof = my_minuit.ndof
print(f"Valore del Q^2: {Q_squared}")
print(f"Gradi di libertà: {N_dof}")
```

I risultati del fit sono contenuti nell'oggetto `my_minuit`

```Python
for par, val, err in zip(my_minuit.parameters, my_minuit.values, my_minuit.errors):
    print(f"{par} = {val:.3f} +/ {err:.3f}")

m_fit = my_minuit.values[0]
q_fit = my_minuit.values[1]
```

Per visualizzare la matrice di covarianza

```Python
print(my_minuit.covariance) # matrice di covarianza
print(my_minuit.covariance[0][1]) # elemento della matrice
print(my_minuit.covariance["m"]["q"]) # accesso ad un elemento usando nomi dei parametri
print(my_minuit.covariance.correlation) # Matrice di correlazione
```

Per visualizzare un sommario di tutte le informazioni sul fit
```Python
display(my_minuit)
```

### Bontà del fit
Nel caso in cui la distribuzione di probabilità degli $ y_{i} $ sia gaussiana, il $ Q_{min}^{2} $ segue una distribuzione di probabilità $ \chi^2(N - k) $, come già detto in precedenza. Si può quindi verificare la bontà del fit tramite un test del $ \chi^2 $, controllando che $ \chi_{rid}^2 \simeq 1 $. Per ottenere la p-value si calcola la probabilità che il risultato sia peggiore di quello ottenuto, integrando $ \chi^2(N - k) $ da $ Q_{min}^{2} $ a $ +\infty $. In Python

```Python
from scipy.stats import chi2
p_value = 1.0 - chi2.cdf(my_minuit.fval, df = my_minuit.ndof)
```